# Create Pipeline in combo w/ GridsearchCV

## Install packages on machine
uncomment to install on machine

In [1]:
# import sys
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install scikit-learn
#!{sys.executable} -m pip install xgboost
#!{sys.executable} -m pip install lightgbm
#!{sys.executable} -m pip install imbalanced-learn

In [3]:
import pandas as pd
import numpy as np
from imblearn.pipeline import Pipeline
from imblearn import FunctionSampler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, KernelPCA, SparsePCA
from sklearn.cluster import FeatureAgglomeration
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator, IterativeImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_regression, \
                                      f_regression, SelectFromModel, GenericUnivariateSelect, \
                                      SequentialFeatureSelector
from sklearn.svm import SVR
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression, LogisticRegression, HuberRegressor, \
                                 RANSACRegressor, BayesianRidge
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import r2_score

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', 500) 

In [4]:
path = ''

In [5]:
df_train = pd.read_csv(path+'X_train.csv')
df_test = pd.read_csv(path+'X_test.csv')
df_y = pd.read_csv(path+'y_train.csv')

In [6]:
df_train.head()

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
0,0.00,10.89,832442.81,20585.54,1028.37,1163779.65,9.20,597900.48,NaN,1144294.32,...,1024198.18,-855.55,12176.07,10.65,10.92,1220.07,8.57,1036263.30,85338.56,103088.66
1,1.00,11.51,832442.90,NaN,1012.62,1028910.54,10.91,597900.46,8127.02,1099166.21,...,1086805.85,-787.40,10493.10,10.59,9.46,917.09,10.23,1007163.22,95695.02,105161.11
2,2.00,11.05,832442.90,20585.51,1003.95,923175.65,9.21,597900.43,10738.09,1027862.70,...,1018532.66,-907.00,10959.52,10.77,10.34,637.03,10.71,1019955.44,80253.30,104177.05
3,3.00,11.64,NaN,NaN,1004.67,945946.09,9.55,597900.45,13524.10,1168144.27,...,1047017.25,-1011.74,16845.31,10.48,10.59,1114.07,10.32,1085441.78,NaN,102746.52
4,4.00,10.41,832442.83,20585.56,NaN,995718.19,8.42,597900.42,12894.07,1063198.89,...,1031009.46,-1025.22,18348.46,NaN,NaN,1230.09,10.25,1024812.19,101815.75,105163.75


## Parameters

In [7]:
scoring = 'r2'
cv = 5
n_jobs = -1 # if not 1 gridsearch does not print verbose output/progress
refit = True

## Helper functions for outlier detection wrapper

In [8]:
def outlier_rejection_isolationforest(X, y):
    """This will be our function used to resample our dataset."""
    model = IsolationForest(max_samples=100, contamination=0.4)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]

def outlier_rejection_LOF(X, y):
    """This will be our function used to resample our dataset."""
    model = LocalOutlierFactor(n_neighbors=80, contamination=0.4, n_jobs=-1)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]

In [17]:
search_space = [ 

    ## commented out means "is running on vastai"

    ### trees regressors
    #{'scaler': [RobustScaler(), QuantileTransformer()],
    # 'imputer': [KNNImputer()],
    # 'imputer__n_neighbors': [20, 300],
    # 'imputer__weights': ['distance', 'uniform'],
    # 'variance': [VarianceThreshold()],
    # 'selector': [SelectKBest()],
    # 'selector__k': [100, 200, 400],
    # #'selector__score_func': [mutual_info_regression, f_regression],
    # 'dimreduction': [SparsePCA(n_jobs=-1)],# KernelPCA(n_jobs=-1)],
    # 'dimreduction__n_components': [0.95, 0.85, 200],
    # 'outliers': [FunctionSampler(func=outlier_rejection_isolationforest, validate=False),
    #              FunctionSampler(func=outlier_rejection_LOF, validate=False)],
    # 'regressor': [lgb.LGBMRegressor(), xgb.XGBRegressor(), RandomForestRegressor()],
    # #'regressor__boosting_type': ['gbdt', 'lgbm'],
    # 'regressor__n_estimators': [100, 500],
    # #'regressor__num_leaves': [2, 31, 50, 100],
    # #'regressor__min_samples_leaf': [1, 5, 10],
    # #'regressor__subsample': [0.7, 1], # only sample fraction of training data before growing trees -> avoid overfitting
    # #'regressor__max_depth ': [-1, 10, 50], # It is necessary to tune num_leaves with the max_depth together.
    # #'regressor__reg_alpha': [0, 1e-2, 10],
    # #'regressor__reg_lambda': [0, 1e-2, 10],
    #},
 
    ### non trees regressors
    #{'scaler': [RobustScaler(), QuantileTransformer()],
    # 'imputer': [KNNImputer()],
    # 'imputer__n_neighbors': [20, 100, 300],
    # #'imputer__weights': ['distance', 'uniform'],
    # 'variance': [VarianceThreshold()],
    # 'selector': [SelectKBest()],
    # 'selector__k': [100, 200, 400],
    # 'selector__score_func': [mutual_info_regression, f_regression],
    # 'dimreduction': [SparsePCA(n_jobs=-1)],# KernelPCA(n_jobs=-1)],
    # 'dimreduction__n_components': [0.95, 0.85, 200],
    # 'outliers': [FunctionSampler(func=outlier_rejection_isolationforest, validate=False),
    #              FunctionSampler(func=outlier_rejection_LOF, validate=False)],
    # 'regressor': [HuberRegressor(), RANSACRegressor(), BayesianRidge()],
    # #'regressor__boosting_type': ['gbdt', 'lgbm'],
    #},

    ## kNN imputer - SVR regressor
    #{'scaler': [RobustScaler(), QuantileTransformer()],
    # 'imputer': [KNNImputer()],
    # 'imputer__n_neighbors': [20, 100, 300],
    # 'imputer__weights': ['distance', 'uniform'],
    # 'variance': [VarianceThreshold()],
    # 'selector': [GenericUnivariateSelect(score_func=f_regression, mode='k_best', param=200)],
    # 'dimreduction': [SparsePCA(n_jobs=-1), KernelPCA(n_jobs=-1)],
    # 'dimreduction__n_components': [0.95, 0.85, 200, None],
    # 'outliers': [FunctionSampler(func=outlier_rejection_isolationforest, validate=False),
    #              FunctionSampler(func=outlier_rejection_LOF, validate=False)],
    # 'regressor': [SVR()],
    # 'regressor__C': [1e-4, 1, 200],
    # 'regressor__epsilon': [1e-4, 1, 10],
    #},
    
    # run local
    ## Forward selection
    {'scaler': [RobustScaler(), QuantileTransformer()],
     'imputer': [KNNImputer()],
     'imputer__n_neighbors': [80, 200],
     'dimreduction': [KernelPCA(n_jobs=-1)],
     'dimreduction__n_components': [0.95, 0.85, 200, None],
     'selector': [SequentialFeatureSelector(estimator=KNeighborsClassifier(n_neighbors=3), \
                                           direction='forward', n_jobs=-1)],
     'selector__n_features_to_select': [50, 100, 200],
     'outliers': [FunctionSampler(func=outlier_rejection_isolationforest, validate=False),
                  FunctionSampler(func=outlier_rejection_LOF, validate=False)],
     'regressor': [RandomForestRegressor(n_estimators=600)],
    },
    # run local
    ## Select from Model
    {'scaler': [RobustScaler(), QuantileTransformer()],
    'imputer': [KNNImputer()],
    'imputer__n_neighbors': [80, 200],
    'variance': [VarianceThreshold()],
    'dimreduction': [KernelPCA(n_jobs=-1)],
    'dimreduction__n_components': [0.95, 0.85, 200, None],
    'selector': [
      SelectFromModel(estimator=LogisticRegression()),
      SelectFromModel(estimator=LinearSVC(penalty="l1"))],
    'outliers': [FunctionSampler(func=outlier_rejection_isolationforest, validate=False),
              FunctionSampler(func=outlier_rejection_LOF, validate=False)],
    'regressor': [RandomForestRegressor(n_estimators=600)]
    }
]

## Design Pipeline

In [18]:
pipeline_outliers_last = Pipeline([
    ('scaler', RobustScaler()),
    ('imputer', KNNImputer()),
    ('variance', None),
    ('selector', SelectKBest()),
    ('dimreduction', None),
    ('outliers', None),
    ('regressor', xgb.XGBRegressor()),
])

In [19]:
pipeline_outliers_first = Pipeline([
    ('scaler', RobustScaler()),
    ('imputer', KNNImputer()),
    ('outliers', None),
    ('selector', SelectKBest()),
    ('dimreduction', None),
    ('regressor', xgb.XGBRegressor()),
])

In [20]:
#for later
pipeline_scale_twice = Pipeline([
    ('scaler_1', RobustScaler()),
    ('imputer', KNNImputer()),
    ('selector', SelectKBest()),
    ('dimreduction', 'passthrough'),
    ('scaler_2', RobustScaler()),
    ('outliers', None),
    ('regressor', xgb.XGBRegressor()),    
]) 
# can set passthrough in Pipeline but not param space

In [21]:
grid_outliers_last = GridSearchCV(estimator=pipeline_outliers_last, param_grid=search_space, cv=cv, verbose=10,
                    n_jobs=n_jobs, scoring=scoring, return_train_score=False, refit=refit)

In [22]:
grid_outliers_first = GridSearchCV(estimator=pipeline_outliers_first, param_grid=search_space, cv=cv, verbose=10,
                    n_jobs=n_jobs, scoring=scoring, return_train_score=False, refit=refit)

In [23]:
X_train = df_train.drop('id', axis=1)
y_train = df_y.drop('id', axis=1).values.ravel()

In [24]:
grid_outliers_last.fit(X_train, y_train)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


KeyboardInterrupt: 

In [ ]:
grid_outliers_first.fit(X_train, y_train)

In [ ]:
best_score = grid_ouliers_last.best_score_
best_estimator_ouliers_last = grid_ouliers_last.best_estimator_
y_pred_1 = best_estimator_ouliers_last.predict(X_test)

In [ ]:
best_score = grid_ouliers_first.best_score_
best_estimator_ouliers_first = grid_ouliers_first.best_estimator_
y_pred_2 = best_estimator_ouliers_first.predict(X_test)

In [ ]:
y_pred_1 = pd.to_csv('y_pred_outlierslast_best.csv', index=False)

In [ ]:
y_pred_2 = pd.to_csv('y_pred_outliersfirst_best.csv', index=False)